# GOParking!
### Goditi la città, noi pensiamo al resto.

Vogliamo un software che: 

-accetti auto **all'ingresso**

    -prendendo targa
    
    -orario di ingresso
    
    -chieda il brand di auto
    
    -indichi un piano (tutte le panda vanno al 5° piano fino a riempimento)

-**lasci uscire una macchina**
    
    -calcolando il totale dovuto fino ad ora compretata

-**aggiorni il database**

In [1]:
#struttura del parcheggio
parking = {
    '1': {'max_capacity':20,
          'cars_in': 7,
          'price_per_hour': 2.5,
          'brands':'all'},

    '2': {'max_capacity':15, 
          'cars_in': 4, 
          'price_per_hour': 3.0, 
          'brands':'all'},

    '3': {'max_capacity':10, 
          'cars_in': 2, 
          'price_per_hour': 4.0, 
          'brands':'all'},

    '4': {'max_capacity':25, 
          'cars_in': 10, 
          'price_per_hour': 1.5, 
          'brands':'all'},
          
    '5': {'max_capacity':10, 
          'cars_in': 5, 
          'price_per_hour': 7.0, 
          'brands':'panda 4x4'},
}

In [2]:
#modulo che ci permette di convertire degli orari scritti come stringa in orario
from datetime import datetime

In [3]:
# dati delle macchine presenti nel parcheggio
cars = {'CA385NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '1'},
        'CA123NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '2'},
        'CA456NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '3'},
        'CA789NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '4'},
        'CA101NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '5'},
        'CA112NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '1'},
        'CA123NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '2'},
        'CA456NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '3'},
        'CA789NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '4'},
        'CA101NN': {'ingresso':  datetime.strptime(f"14/04/2025 10:25", "%d/%m/%Y %H:%M"), 'piano': '5'}
        }

##### Creiamo una lista di piani, dal più costoso al più economico per sapere quale piano suggerire per primo alle non-panda

In [4]:
# Creiamo una lista per capire quale piano è più costoso escludendo il piano della panda
# Questo perchè lo suggeriremo in ordine di pprezzo se non hanno una panda
level_order = ['3', '2', '1', '4']

#### Occupiamoci di come fare a far capire al nostro programma se c'è disponibilità nel piano

In [5]:
def check_level_availability(level): # funzione per capire posti disponibili, controlla solo un piano
    # gli diciamo che c'è la possibilità di entrare in un piano 
    # solo se la capacità massima di quel piano meno le macchine parcheggiateci dentro danno un risultato maggiore di 0
    return parking[level]['max_capacity'] - parking[level]['cars_in'] > 0

In [6]:
# creiamo una funzione per controllare la disponibilità non solo in un piano, ma se c'è almeno un posto nei vari piani
def availability():
    # per ogni livello del parcheggio dimmi quanti posti disponibili ci sono
    for level in parking:
    # se c'è dispobilità, sfruttando la funzione creata prima che controlla la disponibilità per ogni piano, restituiscimi un bool 
     if check_level_availability(level):
        return True

##### Occupiamoci delle domande per fare in modo che non si possano dare le risposte più disparate

In [7]:
# bisogna creare una funzione generale che per tutte le domande controlli le risposte accettate, altrimenti ripete la domanda
def make_question(question, accepted_answer = ['y', 'n', 'q']):
    answer = input(question)
    # gli dobbiamo dire che se la risposta non è una di quelle accettate, che sono quelle inserite in lista
    if not answer in accepted_answer:
        #diciamo che la risposta è sbagliata
        print('Risposta non accettata')
        # allora bisogna rifare la domanda
        answer = make_question(question, accepted_answer = ['y', 'n', 'q'])
    return answer

##### Ora chiediamo prima di tutto se ha una panda, se si controlliamo la disponibilità nel piano 5, altrimenti gli suggeriamo un altro piano sfruttando la lista creata in precedenza per suggerire come prima opzione di piano il più costoso

In [8]:
# creiamo una funziona per suggerire un piano
def suggest_level(panda_answer): 
    #se ha una panda e ci sono posti liberi al 5° piano
    if panda_answer == 'y' and check_level_availability('5'):
        # vai al quinto piano
        return '5'
    # se o non è una panda o non c'è posto al 5° piano
    else:
        # dobbiamo fare un ciclo for per controllare gli altri livelli, cercando di mandarli al più costoso,
        # il ciclo for è sulla lista dei piani che abbiamo ordinato in base al prezzo
        for l in level_order:
            if check_level_availability(l):
                return l


##### Chiediamo la targa

In [9]:
# funzione ricorsiva fino a quando non ci dà una targa valida (4 lettere e tre numeri inserita in  .upper) o quit 
def get_plate():
    plate = input('Digitare numero di targa')
    # gli diciamo di controllare se nella targa le prime due posizioni della targa sono due caratteri alfabetici
    # e dalla seconda posizione in poi (quindi dalla terza) alla penultima (che è 5 perchè si parte a contare da 0) sono numeri
    # e le ultime due sono lettere
    if plate[:2].isalpha() and plate[2:5].isdigit() and plate[-2:].isalpha():
        # inseriamo il modulo per inserirla in maiuscolo per ordinarlo meglio
        return plate.upper()
    return get_plate()

##### Chiediamo l'orario

In [10]:
# funzione per l'orario d'ingresso
def get_time():
    return datetime.now()

##### Dobbiamo inserire dentro 'cars' la macchina che è appena entrata per sapere ora e piano e stabilire la tariffa che dovrà pagare

In [11]:
# definiamo la funzione e diamogli dei parametri che devono rispettare gli stessi parametri del dizionario
# la prima chiave è una targa che ci riporta ad un dizionario che ci fa accedere alle info per quella targa. e per questa targa abbiamo info di ingresso e piano
def add_to_plates(plate, entry_time, level):
    # ora assegnamo alle chiavi di cars i valori
    cars[plate] = {'ingresso': entry_time, 'piano': level}

##### Dobbiamo inserire la macchina nuova che è entrata nel nostro parcheggio

In [12]:
# Dobbiamo incrementare cars_in per aggiungere la macchina nuova al piano in modo che ovviamente faccia numero e possa essere contata la macchina 
# Quello che ci interessa si trova in parking ed è il piano, che è la chiave
def add_to_parking(level):
    # gli stiamo dicendo di entrare in parking, andare alla variabile cars_in e fare l'operazione di conteggio
    parking[level]['cars_in'] += 1

### Codice che implementa tutte le funzioni fatte fino ad ora:

In [ ]:
answer = make_question('Stai entrando?')
# controllo numero di posti disponibili
if answer == 'y' and availability():
        # chiediamo che macchina abbia, in particolare se ha una panda per mandarla al piano 5 se ci sono posti disponibili
        answer = make_question('Hai una panda?')
        # richiamiamo la funzione che suggerisce il piano
        level = suggest_level(answer)
        print(f'Ti è stato assegnato il piano {level}!')
        # inseriamo la funzione per chiedere il numero di targa
        plate = get_plate()
        # inseriamo la funzione che richiama l'orario
        time = get_time()
        # inseriamo la funzione che aggiunge la macchina a cars dopo aver chiesto tutte le info
        # stiamo dicendo che i segnaposto, cioè i valori della funzione add_to_plate, corrispondono alle nostre 
        add_to_plates(plate = plate, entry_time = time, level = level)
        # diciamogli che bisogna aggiungere la macchina al parcheggio
        add_to_parking(level = level)

Risposta non accettata
Risposta non accettata
Risposta non accettata
Ti è stato assegnato il piano 3!


##### Puliamo il codice

In [ ]:
# tutta questo che abbiamo scritto nel codice si può togliere facendo così:
# creo una funzione che all'interno racchiuda tutto quello che ho scritto sopra
def cars_in():
    answer = make_question('Hai una panda?')
    # richiamiamo la funzione che suggerisce il piano
    level = suggest_level(answer)
    print(f'Ti è stato assegnato il piano {level}!')
    # inseriamo la funzione per chiedere il numero di targa
    plate = get_plate()
    # inseriamo la funzione che richiama l'orario
    time = get_time()
    # inseriamo la funzione che aggiunge la macchina a cars dopo aver chiesto tutte le info
    # stiamo dicendo che i segnaposto, cioè i valori della funzione add_to_plate, corrispondono alle nostre 
    add_to_plates(plate = plate, entry_time = time, level = level)
    # diciamogli che bisogna aggiungere la macchina al parcheggio
    add_to_parking(level = level)

### CODICE PULITO:

In [ ]:
answer = make_question('Stai entrando?')
if answer == 'y' and availability():
    cars_in()
    # aggiungiamo un 'oppure' con elif, cioè se non entra esce
elif answer == 'n':
    # riutilizziamo la funzione make question per la targa e inseriamo un modulo di cars come risposta accettata
    # in questo modo mi darà la chiave di cars, che sono le targhe
    plate = make_question('inserisci targa', accepted_answer = cars.keys())
    # ora andiamo a prendere in considerazione l'orario d'ingresso da cars
    entry_time = cars[plate]['ingresso']
    # e il piano da cui viene la macchina sempre da cars
    level = cars[plate]['piano']
    # ora dobbiamo calcolare il prezzo. 
    # Ricordiamo che ogni piano ha la sua tariffa, quindi prima vediamo quanto costa il piano da cui viene la macchina in parking stavolta abbiamo 'price_per_hour':
    price_per_hour = parking[level]['price_per_hour']
    # vediamo l'ora corrente, cioè a che ora sta uscendo. get_time infatti mi dice l'ora sul momento
    current_time = get_time()
    # facciamo l'operazione togliendo dall'orario di uscita l'orario di entrata
    # però questo calcolo non è esattamente corretto, noi dobbiamo dirgli che appena entra scatta un'ora e deve pagare la prima ora, poi quando parte la seconda
    # deve pagare la seconda e così via. Quindi l'orario va calcolato in maniera più precisa
    duration = current_time - entry_time
    print(duration)
    # diciamogli come calcolare le ore intere e aggiungiamogliene una che è quella da pagare appena si entra
    ore_intere = int(duration.total_seconds()/3600) + 1
    # ora definiamo il prezzo totale, dove moltiplichiamo le ore intere per la tariffa oraria
    total_price = ore_intere * price_per_hour



### Completiamo il flusso d'uscita, creiamo una funzione car_out

In [ ]:
# Funzione per completare l'uscita di un'auto dal parcheggio

def car_out(plate):
    # 1. Verifica se la targa è presente
    if plate not in cars:
        print("Targa non trovata nel parcheggio.")
        return

    # Recupera dati di ingresso e piano
    entry_time = cars[plate]['ingresso']
    level = cars[plate]['piano']
    price_per_hour = parking[level]['price_per_hour']

    # Calcola durata e costo
    current_time = get_time()
    duration = current_time - entry_time
    ore_intere = int(duration.total_seconds() / 3600) + 1
    total_price = ore_intere * price_per_hour

    # Stampa ricevuta di pagamento
    print(f"Auto con targa {plate} ha sostato per {ore_intere} ore.")
    print(f"Totale da pagare: {total_price:.2f} euro.")

    # Aggiorna il database: decrementa cars_in e rimuovi la targa
    parking[level]['cars_in'] -= 1
    del cars[plate]